In [1]:
import random
import numpy as np
import sys
import time

    
def activationSigmoid(n):
    
    return 1.0/(1.0+np.exp(-n))

def activationSoftmax(n):

    return np.exp(n)/np.sum(np.exp(n), axis=0, keepdims=True)

def activationSigmoidDerivative(z):
    
    return activationSigmoid(z)*(1-activationSigmoid(z))   

    
def backwardLoss(predY, trueY):

    return (predY-trueY)
    
def clusterSync(cluster, derivative):

    return [i+j for i, j in zip(cluster, derivative)]
    


class NeuralNetwork(object):

    def __init__(self, layerCount,neuronCount):
        
        self.layerCount = layerCount
        self.neuronCount = neuronCount
        self.load = [np.random.randn(y, 1) for y in self.neuronCount[1:]]
        self.cost = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.neuronCount[:-1], self.neuronCount[1:])]
        



    def forwardPass(self, input):
  
        activations = [input]  
        beforeActivation=[]
        count=0
        
        for l, c in zip(self.load, self.cost):
            count=count+1
            t=np.dot(c, input)+l
            beforeActivation.append(t)
            if count<3:
                input = activationSigmoid(t)
            else:
                input = activationSoftmax(t)
            activations.append(input)
            
        return input,activations,beforeActivation
        
    def forwardPassS(self, input):
      
        count=0
        
        for l, c in zip(self.load, self.cost):
            count=count+1
            temp=np.dot(c, input)+l
            if count<3:
                input = activationSigmoid(temp)
            else:
                input = activationSoftmax(temp)
           
        return input
            
   
       
    def backpropagation(self, x, y,activations,beforeActivation):
    
        derivativeC = [np.zeros(c.shape) for c in self.cost]
        derivativeL = [np.zeros(l.shape) for l in self.load]
        
             
        for l in range(1, self.layerCount):
            if l ==1:
                lossBack = backwardLoss(activations[-l], y)
                derivativeC[-l] = np.dot(lossBack, activations[-l-1].T)
                derivativeL[-l] = lossBack
            else:
                lossBack = np.dot(self.cost[-l+1].T, lossBack) * activationSigmoidDerivative(beforeActivation[-l])
                derivativeC[-l] = np.dot(lossBack, activations[-l-1].T)
                derivativeL[-l] = lossBack
            
        return (derivativeL, derivativeC)


    def trainModel(self, trainingData, testInputs, epochs, size ,r):
 
        trainingData = list(trainingData)
        l = len(trainingData)
        result = []
       
        for ep in range(epochs):
            spacedValue = np.arange(len(trainingData))
            random.shuffle(trainingData)
            np.random.shuffle(spacedValue)
            clusters = [trainingData[k:k+size] for k in range(0, l, size)]
                     
            for cluster in clusters:
                clusterC = [np.zeros(c.shape) for c in self.cost]
                clusterL = [np.zeros(l.shape) for l in self.load]             
                
                for x, y in cluster:
                    predictionY,activations,beforeActivation=self.forwardPass(x)
                    derivativeL, derivativeC = self.backpropagation(x, y,activations,beforeActivation)
                    z=np.array(derivativeC)+np.array(clusterC)
                    clusterL=clusterSync(clusterL,derivativeL)
                    clusterC=clusterSync(clusterC,derivativeC)
                                    
                self.cost = [c-(r/len(cluster))*wc for c, wc in zip(self.cost, clusterC)]
                self.load  = [l-(r/len(cluster))*wl for l, wl in zip(self.load, clusterL)]     
       
        for x in testInputs:
            result.append(np.argmax(self.forwardPassS(x)))    
        return result
    
    
def writeOutput(result):
    with open("test_predictions.csv",'w') as file:
        for res in result:
            file.write(str(res))
            file.write("\n")
    file.close()
    
def vectorizedResult(i):
    ep = np.zeros((10, 1))
    ep[i] = 1.0
    return ep   
    

def formatInput(trainingDataFilename,trainingLabelsFilename,testingDataFilename):
   
    data = np.genfromtxt(trainingDataFilename,delimiter=',')
    trainingDataInputs = [np.reshape((x/255), (784, 1)) for x in data]
    
    data = np.genfromtxt(trainingLabelsFilename,delimiter=',')
    trainingResults = [vectorizedResult(int(y)) for y in data]
    
    trainingData = zip(trainingDataInputs, trainingResults)
    
    data = np.genfromtxt(testingDataFilename,delimiter=',')
    testDataInputs = [np.reshape((x/255), (784, 1)) for x in data]
    
    return trainingData,testDataInputs



trainingDataFilename="train_image.csv"
trainingLabelsFilename="train_label.csv"
testingDataFilename="test_image.csv" 
trainingData,testInputs=formatInput(trainingDataFilename,trainingLabelsFilename,testingDataFilename)

trainingData = list(trainingData)
testInputs = list(testInputs)
n = NeuralNetwork(4,[784, 200,32, 10])
result= n.trainModel(trainingData, testInputs,15, 10, 0.2)

writeOutput(result)


/var/folders/39/4tr3skfn5tx_3qrdyzmdk9480000gn/T/ipykernel_56792/2169063650.py:115: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  z=np.array(derivativeC)+np.array(clusterC)
